获取逻辑
1.依赖环境设置与模块加载

- 通过代码安装了所需的 Python 包，包括 tushare、python-dotenv 和 pymysql。
使用 dotenv 加载 .env 文件中的 API Token，确保数据访问的安全性和可配置性。

2. 数据获取

- 通过 tushare 提供的 pro.daily 方法，下载从 2021年1月1日 至 2024年1月1日 的日线数据，涵盖全部交易所的股票交易记录。
- 利用 API Token 配置 Tushare 的客户端接口。

3. 停牌数据查询

- 定义了 query_suspension(stock_code) 函数，用于查询指定股票的停牌信息，查询时间范围为 2021年1月1日 至 2024年1月1日。
- 遍历获取到的所有股票代码（ts_code），通过上述函数获取停牌数据，并逐步合并到 all_suspension 数据框中。
- 最终将停牌数据保存为 suspension_data.csv 文件。

4. 日期转换

- 将 trade_date 列格式化为 Pandas 的日期格式，以便后续进行时间序列分析

In [6]:
#环境
!pip install tushare
!pip install python-dotenv
!pip install pymysql

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [1]:
#调用库
import tushare as ts
from dotenv import load_dotenv
import pandas as pd 
from tqdm import tqdm
import os
import pymysql

/Users/elvisyan/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
#读取token
load_dotenv()
token = os.getenv("API_TOKEN_TUSHARE")
print(token)
ts.set_token(token)
pro = ts.pro_api()

aa1720cccc1acb18523702e914ca4352428811e10fdf9b01af9d7c0c


In [10]:
stock_list = pro.stock_basic(exchange='', list_status='L', fields='ts_code,name')
print(f"Total stocks fetched: {len(stock_list)}")

Total stocks fetched: 5384


In [11]:
# Create an empty DataFrame to store the results
all_stock_data = pd.DataFrame()
# Loop through each stock code and fetch daily data
for i, ts_code in enumerate(stock_list['ts_code']):
    print(f"Fetching data for {ts_code} ({i+1}/{len(stock_list)})...")
    try:
        # Fetch daily data for the stock
        daily_data = pro.daily(ts_code=ts_code, start_date='20210101', end_date='20240101')
        
        # Append the data to the main DataFrame
        all_stock_data = pd.concat([all_stock_data, daily_data], ignore_index=True)
    except Exception as e:
        print(f"Error fetching data for {ts_code}: {e}")
    # To avoid hitting the API rate limit, add a delay between requests
    import time
    time.sleep(0.5)  # Adjust delay as necessary

Fetching data for 000001.SZ (1/5384)...
Fetching data for 000002.SZ (2/5384)...
Fetching data for 000004.SZ (3/5384)...
Fetching data for 000006.SZ (4/5384)...
Fetching data for 000007.SZ (5/5384)...
Fetching data for 000008.SZ (6/5384)...
Fetching data for 000009.SZ (7/5384)...
Fetching data for 000010.SZ (8/5384)...
Fetching data for 000011.SZ (9/5384)...
Fetching data for 000012.SZ (10/5384)...
Fetching data for 000014.SZ (11/5384)...
Fetching data for 000016.SZ (12/5384)...
Fetching data for 000017.SZ (13/5384)...
Fetching data for 000019.SZ (14/5384)...
Fetching data for 000020.SZ (15/5384)...
Fetching data for 000021.SZ (16/5384)...
Fetching data for 000025.SZ (17/5384)...
Fetching data for 000026.SZ (18/5384)...
Fetching data for 000027.SZ (19/5384)...
Fetching data for 000028.SZ (20/5384)...
Fetching data for 000029.SZ (21/5384)...
Fetching data for 000030.SZ (22/5384)...
Fetching data for 000031.SZ (23/5384)...
Fetching data for 000032.SZ (24/5384)...
Fetching data for 000034.

/var/folders/by/qpw66qgj0ll1h0z_yglfwpfh0000gn/T/ipykernel_45816/3763635231.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stock_data = pd.concat([all_stock_data, daily_data], ignore_index=True)


Fetching data for 001278.SZ (461/5384)...
Fetching data for 001279.SZ (462/5384)...


/var/folders/by/qpw66qgj0ll1h0z_yglfwpfh0000gn/T/ipykernel_45816/3763635231.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stock_data = pd.concat([all_stock_data, daily_data], ignore_index=True)


Fetching data for 001282.SZ (463/5384)...
Fetching data for 001283.SZ (464/5384)...
Fetching data for 001286.SZ (465/5384)...
Fetching data for 001287.SZ (466/5384)...
Fetching data for 001288.SZ (467/5384)...
Fetching data for 001289.SZ (468/5384)...
Fetching data for 001296.SZ (469/5384)...
Fetching data for 001298.SZ (470/5384)...
Fetching data for 001299.SZ (471/5384)...
Fetching data for 001300.SZ (472/5384)...
Fetching data for 001301.SZ (473/5384)...
Fetching data for 001306.SZ (474/5384)...
Fetching data for 001308.SZ (475/5384)...
Fetching data for 001309.SZ (476/5384)...
Fetching data for 001311.SZ (477/5384)...
Fetching data for 001313.SZ (478/5384)...
Fetching data for 001314.SZ (479/5384)...
Fetching data for 001316.SZ (480/5384)...
Fetching data for 001317.SZ (481/5384)...
Fetching data for 001318.SZ (482/5384)...
Fetching data for 001319.SZ (483/5384)...
Fetching data for 001322.SZ (484/5384)...
Fetching data for 001323.SZ (485/5384)...
Fetching data for 001324.SZ (486/5

...

Fetching data for 920118.BJ (5382/5384)...


/var/folders/by/qpw66qgj0ll1h0z_yglfwpfh0000gn/T/ipykernel_45816/3763635231.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stock_data = pd.concat([all_stock_data, daily_data], ignore_index=True)


Fetching data for 920128.BJ (5383/5384)...


/var/folders/by/qpw66qgj0ll1h0z_yglfwpfh0000gn/T/ipykernel_45816/3763635231.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stock_data = pd.concat([all_stock_data, daily_data], ignore_index=True)


Fetching data for 689009.SH (5384/5384)...


In [13]:
all_stock_data.to_csv('data/rawdata.csv')

In [15]:
all_stock_data['trade_date'] = pd.to_datetime(all_stock_data['trade_date'])
all_stock_data= all_stock_data.sort_values(by=['ts_code', 'trade_date'])
all_stock_data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
726,000001.SZ,2021-01-04,19.10,19.10,18.44,18.60,19.34,-0.74,-3.8263,1554216.43,2891682.312
725,000001.SZ,2021-01-05,18.40,18.48,17.80,18.17,18.60,-0.43,-2.3118,1821352.10,3284606.913
724,000001.SZ,2021-01-06,18.08,19.56,18.00,19.56,18.17,1.39,7.6500,1934945.12,3648521.909
723,000001.SZ,2021-01-07,19.52,19.98,19.23,19.90,19.56,0.34,1.7382,1584185.30,3111274.625
722,000001.SZ,2021-01-08,19.90,20.10,19.31,19.85,19.90,-0.05,-0.2513,1195473.22,2348316.363
...,...,...,...,...,...,...,...,...,...,...,...
3443709,873833.BJ,2023-12-25,13.70,14.20,13.67,14.20,13.67,0.53,3.8771,22215.96,30991.415
3443708,873833.BJ,2023-12-26,14.10,14.49,13.76,14.07,14.20,-0.13,-0.9155,16599.72,23500.948
3443707,873833.BJ,2023-12-27,14.03,14.65,14.03,14.33,14.07,0.26,1.8479,19229.63,27533.313
3443706,873833.BJ,2023-12-28,14.23,14.88,14.04,14.82,14.33,0.49,3.4194,27745.51,40212.405


In [54]:
#读取2021年1月1日 至 2024年1月1日 之内的停牌数据
def query_suspension(stock_code):
    try:
        suspension_data = pro.suspend(ts_code=stock_code, start_date='20210101', end_date='20240101')
        return suspension_data
    except Exception as e:
        print(f"Error fetching data for {stock_code}: {e}")
        return pd.DataFrame() 

all_suspension = pd.DataFrame()
for stock in tqdm(all_data['ts_code'].unique()):
    suspension_results = query_suspension(stock)
    all_suspension = pd.concat([suspension_results, all_suspension])
all_suspension

100%|██████████| 5328/5328 [1:25:42<00:00,  1.04it/s]


,ts_code,suspend_date,resume_date,suspend_reason
0,001202.SZ,20210429,None,盘中成交价较开盘价首次上涨达到或超过10%
0,000573.SZ,20130830,20130902,重要事项未公告
1,000573.SZ,20130829,None,重要事项未公告
2,000573.SZ,20130828,None,重要事项未公告
3,000573.SZ,20130827,None,重要事项未公告
...,...,...,...,...
102,002276.SZ,20090804,None,公布交易异常波动
103,002276.SZ,20090803,None,公布交易异常波动
104,002276.SZ,20090731,None,公布交易异常波动
105,002276.SZ,20090717,None,公布其它公告


In [55]:
all_suspension.to_csv('data/suspension_data.csv')

In [16]:
all_suspension = pd.read_csv('data/suspension_data.csv')

In [17]:
#从原数据里过滤停牌数据
all_suspension['suspend_date'] = pd.to_datetime(all_suspension['suspend_date'])
all_suspension['resume_date'] = pd.to_datetime(all_suspension['resume_date'])
def is_not_suspended(row, suspension_df):
    ts_code = row['ts_code']
    #print(ts_code)
    trade_date = row['trade_date']
    suspensions = suspension_df[suspension_df['ts_code'] == ts_code]
    for _, suspension in suspensions.iterrows():
        #print(trade_date, suspension['suspend_date'])
        if (suspension['suspend_date'] == trade_date) or (suspension['suspend_date'] <= trade_date <= suspension['resume_date']):
            #print('suspension detected')
            return False  
    return True  
filtered_data = all_stock_data[all_stock_data.apply(is_not_suspended, axis=1, suspension_df=all_suspension)]

In [21]:
#最终数据展示
filtered_data = filtered_data.reset_index().drop('index', axis=1)
filtered_data

,level_0,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,0,000001.SZ,2021-01-04,19.10,19.10,18.44,18.60,19.34,-0.74,-3.8263,1554216.43,2891682.312
1,1,000001.SZ,2021-01-05,18.40,18.48,17.80,18.17,18.60,-0.43,-2.3118,1821352.10,3284606.913
2,2,000001.SZ,2021-01-06,18.08,19.56,18.00,19.56,18.17,1.39,7.6500,1934945.12,3648521.909
3,3,000001.SZ,2021-01-07,19.52,19.98,19.23,19.90,19.56,0.34,1.7382,1584185.30,3111274.625
4,4,000001.SZ,2021-01-08,19.90,20.10,19.31,19.85,19.90,-0.05,-0.2513,1195473.22,2348316.363
...,...,...,...,...,...,...,...,...,...,...,...,...
3444492,3444492,873833.BJ,2023-12-25,13.70,14.20,13.67,14.20,13.67,0.53,3.8771,22215.96,30991.415
3444493,3444493,873833.BJ,2023-12-26,14.10,14.49,13.76,14.07,14.20,-0.13,-0.9155,16599.72,23500.948
3444494,3444494,873833.BJ,2023-12-27,14.03,14.65,14.03,14.33,14.07,0.26,1.8479,19229.63,27533.313
3444495,3444495,873833.BJ,2023-12-28,14.23,14.88,14.04,14.82,14.33,0.49,3.4194,27745.51,40212.405


In [19]:
#保存数据至本地
filtered_data.to_csv('data/filtered_data.csv')

In [4]:
#保存数据至本地MySQL数据库中
db_config = {
    'host': 'localhost',
    'user': 'root',  
    'password': '',
    'database': 'equity_data'
}
connection = pymysql.connect(**db_config)
try:
    filtered_data = pd.read_csv('data/filtered_data.csv')
    cursor = connection.cursor()
    for _, row in filtered_data.iterrows():
        sql = """
        INSERT INTO stock_data (ts_code, trade_date, open, high, low, close, pre_close, `change`, pct_chg, vol, amount)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql, (
            row['ts_code'], row['trade_date'], row['open'], row['high'], row['low'],
            row['close'], row['pre_close'], row['change'], row['pct_chg'],
            row['vol'], row['amount']
        ))
    connection.commit()
    print("Data saved.")
except Exception as e:
    print("An error occurred:", e)
finally:
    # Close the connection
    connection.close()

Data saved.
